In [ ]:
#그동안 짜주신 코드들 바탕으로 합쳐서 기사 제목, 발행일, 내용 요약, 키워드를 한눈에 볼 수 있는 코드 작성해두었습니다.
#요약 내용이랑 키워드 퀄리티 올리는 작업 필요

In [6]:
# 필요한 라이브러리 설치
!pip install newsapi-python
!pip install requests beautifulsoup4
!pip install sumy
!pip install konlpy
!pip install nltk scikit-learn

In [7]:
import requests
from bs4 import BeautifulSoup
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# 네이버 API 키 설정
client_id = 'xIpH54vcwURuvzz8t0vl'
client_secret = 'dfHi2SP3CQ'

# API 요청 URL
url = 'https://openapi.naver.com/v1/search/news.json'

# 카테고리 설정
categories = {
    '경제': '경제',
    'IT': 'IT',
    '과학': '과학',
    '사회': '사회',
    '문화': '문화',
    '스포츠': '스포츠',
    '건강': '건강'
}

# 사용자로부터 카테고리 선택 받기
print("카테고리를 선택하세요: 경제, IT, 과학, 사회, 문화, 스포츠, 건강")
user_choice = input("선택한 카테고리: ")

# 유효한 카테고리인지 확인
if user_choice in categories:
    selected_category = categories[user_choice]
else:
    print("유효하지 않은 카테고리입니다. 기본적으로 '경제' 카테고리를 선택합니다.")
    selected_category = '경제'

# 파라미터 설정
params = {
    'query': selected_category,  # 검색어
    'display': 5,            # 5개의 뉴스 가져오기
    'sort': 'date'           # 날짜 기준 정렬
}

# 헤더 설정
headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret
}

# API 요청
response = requests.get(url, params=params, headers=headers)

# 응답을 JSON 형태로 파싱
data = response.json()

# 뉴스 정보 출력 및 기사 내용 추출
if 'items' in data:
    articles = data['items']
    for i, article in enumerate(articles, 1):
        title = article['title']
        description = article['description']
        link = article['link']
        pub_date = article['pubDate']

        print(f"뉴스 {i}:")
        print(f"제목: {title}")
        print(f"URL: {link}")
        print(f"발행일: {pub_date}")

        # URL에서 기사 텍스트 추출
        extracted_text = extract_text_from_url(link)

        if extracted_text:
            # 요약 기능
            parser = PlaintextParser.from_string(extracted_text, Tokenizer("korean"))
            summarizer = TextRankSummarizer()
            summary = summarizer(parser.document, 3)
            summary_text = " ".join(str(sentence) for sentence in summary)

            print("내용 요약:")
            for sentence in summary:
                print(sentence)

            # 키워드 추출 기능
            okt = Okt()

            def extract_keywords(text, num_keywords=5):
                text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\\s]', '', text)
                nouns = okt.nouns(text)
                if not nouns:
                    return []
                vectorizer = TfidfVectorizer(max_features=num_keywords)
                tfidf_matrix = vectorizer.fit_transform([' '.join(nouns)])
                feature_names = vectorizer.get_feature_names_out()
                sorted_items = sorted(zip(vectorizer.idf_, feature_names))
                keywords = [item[1] for item in sorted_items[:num_keywords]]
                return keywords

            keywords = extract_keywords(summary_text, num_keywords=5)

            print(f"키워드: {keywords}")
        else:
            print("기사를 추출할 수 없습니다.")
        print("-" * 80)
else:
    print("뉴스를 가져오는데 실패했습니다.")

카테고리를 선택하세요: 경제, IT, 과학, 사회, 문화, 스포츠, 건강
선택한 카테고리: 경제
뉴스 1:
제목: 대구시, 추석 명절 물가 잡는다···33개 품목 선정해 집중 관리
URL: https://www.dnews.co.kr/uhtml/view.jsp?idxno=202408281118400110971
발행일: Wed, 28 Aug 2024 11:42:00 +0900
내용 요약:
대구시, 추석 명절 물가 잡는다 ···33 개 품목 선정해 집중 관리 - 대한 경제 본문내용 바로 가기 구독신청 기사 제보 입찰 정보 판매도 서 대한 경제 i 신문 초판 회원 가입 내정보 로그 인 로그 아웃 11/02 /2020 월요일 건설· 부동산 증권 산업 금융 경제 정치 사회 레저· 문화 피플 오피니언 전국 대한 경제 TV search search 닫기 건설산업 정책/ 제도 공사계약 전문/ 설비 설계/ 감리 엔지니어링 기술 자재 인테리어 전기/ 통신/ 소방 에너지 IT/ 과학 국토개발 노동/ 안전 산업 산업 일반 재계 가전 자동차 반도체 정유/ 화학 철강/ 조선 항공/ 해운 유통 제약/ 바이오 IT/ 통신/ 방송 게임/ 엔 터 부동산 분양 도시 정비 개발 재테크 부동산시장 경제 경제정책 금융 기업 증권 /IB 종목 뉴스 공시 정치 대통령 실 국회/ 정당 외교/ 통일/ 국방 정치 일반 사회 사회 일반 법조 전국 수도권 지방 오피니언 사설 시론/ 논단 데스크 칼럼 기고/ 칼럼 기자 수첩 마음의 창 세상만 사 Q&A 기획 연재 기획/ 특집 이슈 추적 연재 연구기관 리포트 해외 뉴스 해외건설 국제정치/ 사회 국제경제 국제산업 국제금융 피플 인터뷰 게시판 인사 결혼 부음 레저/ 문화 스포츠 건강/ 레저 BOOK/ 문화 포토 대구시, 추석 명절 물가 잡는다 ···33 개 품목 선정해 집중 관리 기사 입력 2024-08-28 11:26 :27 폰트 크기 변경 대구시가 27일 추석 성수품 가격 안정을 위해 유통업체· 관계기관 관계자들과 대책회의를 열고 있다.
△ 농수축 산물 직거래 장터를